In [1]:
#Mounting the Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#Changing working directory
%cd /content/gdrive/My Drive/ML-in-colab/Sentiment_analysis_amazon_fine_food_review

/content/gdrive/My Drive/ML-in-colab/Sentiment_analysis_amazon_fine_food_review


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/ML-in-colab"

In [5]:
#downloading kaggle dataset
!kaggle datasets download -d snap/amazon-fine-food-reviews -p data

 97% 235M/242M [00:01<00:00, 140MB/s]
100% 242M/242M [00:01<00:00, 140MB/s]


In [6]:
#we can check the content by ls command
!ls data

amazon-fine-food-reviews.zip


In [15]:
#Unziping the file into data folder
!unzip data/\*.zip -d data

Archive:  data/amazon-fine-food-reviews.zip
  inflating: data/Reviews.csv        
  inflating: data/database.sqlite    
  inflating: data/hashes.txt         


In [16]:
#checking the file content
!ls data

amazon-fine-food-reviews.zip  database.sqlite  hashes.txt  Reviews.csv


In [17]:
#removing zip file from data folder
!rm data/*.zip

In [18]:
!ls data

database.sqlite  hashes.txt  Reviews.csv


In [ ]:
#Dowloading required libraries


In [24]:
#importing required libraries
import pandas as pd
import re
from bs4 import BeautifulSoup
import csv

In [20]:
#Reading the file
file = pd.read_csv('data/Reviews.csv')

In [25]:
#Checking the number of total reviews
reviews = 0
with open('data/Reviews.csv') as file:
  reader = csv.reader(file)
  for row in reader:
    reviews +=1
print("Total number of Reviews is: {}".format(reviews))

Total number of Reviews is: 568455


In [21]:
file.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
#testing dataset
#tdf = file[:100]

In [22]:
#keeping the selected columns in DF
data = file[['Text','Score']]
data.head()
#we are experimenting with text only now we will later take the Summary column also

,Text,Score
0,I have bought several of the Vitality canned d...,5
1,Product arrived labeled as Jumbo Salted Peanut...,1
2,This is a confection that has been around a fe...,4
3,If you are looking for the secret ingredient i...,2
4,Great taffy at a great price. There was a wid...,5


In [ ]:
pd.set_option('display.max_rows', None)

In [33]:
#We have '5' scores for the reviews
#We coinsider '1' & '2' as Negative Reviews
#we coinsider '4' & '5' as Positive Reviews
data['Score'] = data['Score'].map({1:0, 2:0, 4:1, 5:1})
#We are remove Reviews with Score '3' bcz they are neighter Positive or negative,
#they are coinsidered to be Neutral
data = data[data['Score']!=3]

In [34]:
#tdf['score'] = tdf['Score'].map({1:0, 2:0, 4:1, 5:1})
#tdf.head(100)
print("total no of reviews now: {}".format(len(data)))

total no of reviews now: 525814


In [35]:
#converting float to int
data['Score'] = data['Score'].astype(int)
data.head()

,Text,Score
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,1
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,1


In [36]:
#tdf = tdf[['Text','score']]
#tdf.head()

In [ ]:
# saving the whole preprocess data
#file = file[file['Score']!=3]
#file['Score'] = file['Score'].map({1:0, 2:0, 4:1, 5:1})
#file['Score'] = file['Score'].astype(int)
#file.drop(labels='Score', axis=1, inplace= True)


Text Preprocessing

In [37]:
#Defining StopWords
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [38]:
#unwanted_words = ['b','c','d'...]

In [39]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def text_preprocess(df, text_column_to_preprocess):

  from tqdm import tqdm
  preprocessed_reviews = []

  for sentance in tqdm(df[text_column_to_preprocess].values):
      sentance = re.sub(r"http\S+", "", sentance)
      sentance = BeautifulSoup(sentance, 'lxml').get_text()
      sentance = decontracted(sentance)
      sentance = re.sub("\S*\d\S*", "", sentance).strip()
      sentance = re.sub('[^A-Za-z]+', ' ', sentance)
      # https://gist.github.com/sebleier/554280
      # now using stopwords as of now
      sentance = ' '.join(e.lower() for e in sentance.split()) # if e.lower() not in stopwords)
      preprocessed_reviews.append(sentance.strip())
  
  df[text_column_to_preprocess] = preprocessed_reviews
  return df

In [40]:
preprocessed_reviews = text_preprocess(data, 'Text')

100%|██████████| 525814/525814 [03:56<00:00, 2219.08it/s]


In [41]:
preprocessed_reviews.sample()

,Text,Score
182195,i have been using this product for more that y...,1


In [53]:
#no of Posive recviews
print("Positive Reviews:",len(preprocessed_reviews[preprocessed_reviews['Score']==1]))

Positive Reviews: 443777


In [54]:
#no of negative recviews
print("Negative Reviews:",len(preprocessed_reviews[preprocessed_reviews['Score']==0]))

Negative Reviews: 82037


We can see its a highly imbalance data so we will add more negative data by duplicating the existing negative reviews

In [55]:
# creating Negative Reviews Df for adding
negative_review_df = preprocessed_reviews[preprocessed_reviews['Score']==0]

In [57]:
#Adding back to the dataframe
preprocessed_reviews = preprocessed_reviews.append([negative_review_df,negative_review_df,negative_review_df,negative_review_df], ignore_index=True)

In [59]:
# shuffle the DataFrame rows
preprocessed_reviews = preprocessed_reviews.sample(frac = 1)

In [60]:
preprocessed_reviews.to_csv('data/preprocessed_reviews.csv', index=False )

Preprocessing Done


In [61]:
len(preprocessed_reviews[preprocessed_reviews['Score']==0])

410185

In [62]:
len(preprocessed_reviews[preprocessed_reviews['Score']==1])

443777

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
final_count = count_vect.fit_transform(file['title'].values)

In [ ]:
final_count_df = pd.DataFrame( final_count.toarray(), columns = count_vect.get_feature_names() )
final_count_df

,00,000,001,002,005,01,015,017,0177,019,02,0201,026,03,0354,04,047,05,051,06,062,07,0702,08,09,096,0994,10,100,1000,1001,1002,101,1014,102,103,1031,104,105,1061,...,中华传染病杂志,中华儿科杂志,中华内分泌代谢杂志,中华创伤杂志,中华医学科研管理杂志,中华医院管理杂志,中华危重病急救医学,中华器官移植杂志,中华围产医学杂志,中华外科杂志,中华实用儿科临床杂志,中华实验和临床病毒学杂志,中华实验眼科杂志,中华心血管病杂志,中华急诊医学杂志,中华放射学杂志,中华检验医学杂志,中华流行病学杂志,中华消化杂志,中华烧伤杂志,中华神经科杂志,中华肝脏病杂志,中华肿瘤杂志,中华胃肠外科杂志,中华预防医学杂志,中国小儿急救医学,中国胸心血管外科临床杂志,中国药房,器官移植,国际眼科杂志,感染爆発を招いた2つの病魔が文政権を追い込む,新型冠状病毒肺炎流行期肺部手术的初步建议,新型冠状病毒肺炎疫情下医学院校开展医学科普的思考,新型冠状病毒肺炎疫情期间肺癌患者临床管理,湖北省及中国其他地区covid,热带医学杂志,羌跖汤,药学学报,重庆医学,韓国
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51389,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
51390,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import numpy as np
#np.shape(a)

In [ ]:
len(file)

51392

In [ ]:
totalwords = set()
for sen in file['title'].tolist():
  for word in sen.split():
    totalwords.add(word)

In [ ]:
len(totalwords)

37236

In [ ]:
text = ['this is rama','I ma sita','i am twinkle','use any other sentence','i will do it']
dd = count_vect.fit_transform( text )

In [ ]:
dddf = pd.DataFrame( dd.toarray(), columns = count_vect.get_feature_names()) 
dddf

,am,any,do,is,it,ma,other,rama,sentence,sita,this,twinkle,use,will
0,0,0,0,1,0,0,0,1,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,1,0,1,0,0,0,1,0
4,0,0,1,0,1,0,0,0,0,0,0,0,0,1


In [ ]:
count_vect.get_feature_names()

['am',
 'any',
 'do',
 'is',
 'it',
 'ma',
 'other',
 'rama',
 'sentence',
 'sita',
 'this',
 'twinkle',
 'use',
 'will']

In [ ]:
#tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer()
tf_idf_value = tf_idf.fit_transform(text)
tf_idf_value.toarray()

array([[0.        , 0.        , 0.        , 0.57735027, 0.        ,
        0.        , 0.        , 0.57735027, 0.        , 0.        ,
        0.57735027, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.70710678, 0.        , 0.        , 0.        , 0.70710678,
        0.        , 0.        , 0.        , 0.        ],
       [0.70710678, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        ],
       [0.        , 0.5       , 0.        , 0.        , 0.        ,
        0.        , 0.5       , 0.        , 0.5       , 0.        ,
        0.        , 0.        , 0.5       , 0.        ],
       [0.        , 0.        , 0.57735027, 0.        , 0.57735027,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.57735027]])

In [ ]:
np.shape(tf_idf_value)

(5, 14)

In [ ]:
tf_idf_value_df = pd.DataFrame( tf_idf_value.toarray(), columns=tf_idf.get_feature_names())
tf_idf_value_df

,am,any,do,is,it,ma,other,rama,sentence,sita,this,twinkle,use,will
0,0.000000,0.0,0.00000,0.57735,0.00000,0.000000,0.0,0.57735,0.0,0.000000,0.57735,0.000000,0.0,0.00000
1,0.000000,0.0,0.00000,0.00000,0.00000,0.707107,0.0,0.00000,0.0,0.707107,0.00000,0.000000,0.0,0.00000
2,0.707107,0.0,0.00000,0.00000,0.00000,0.000000,0.0,0.00000,0.0,0.000000,0.00000,0.707107,0.0,0.00000
3,0.000000,0.5,0.00000,0.00000,0.00000,0.000000,0.5,0.00000,0.5,0.000000,0.00000,0.000000,0.5,0.00000
4,0.000000,0.0,0.57735,0.00000,0.57735,0.000000,0.0,0.00000,0.0,0.000000,0.00000,0.000000,0.0,0.57735


In [ ]:
######################################

In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [ ]:
#!wget https://thigm85.github.io/data/cord19/cord19-query-title-label.csv

In [ ]:
import pandas as pd
file = pd.read_csv('cord19-query-title-label.csv')

In [ ]:
file['X'] = file['query'] +':'+ file['title']
file = file[['X','label']]
file.head()

,X,label
0,coronavirus origin:Monophyletic Relationship b...,1
1,coronavirus origin:Comprehensive overview of C...,1
2,"coronavirus origin:The SARS, MERS and novel co...",1
3,coronavirus origin:Evidence for zoonotic origi...,1
4,coronavirus origin:Deadly virus effortlessly h...,1


In [ ]:
X = embed( file['X'].values)

In [ ]:
print(type(X))
X.shape

<class 'numpy.ndarray'>


(51392, 250)

In [ ]:
X_LSTM = np.reshape(X, X.shape + (1,))
X_LSTM.shape

(51392, 250, 1)

In [ ]:
Y = file['label'].values

In [ ]:
Y.shape

(51392,)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(250)))
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 25)                6275      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 6,301
Trainable params: 6,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X.shape[1:]

(250,)

In [ ]:
model.compile( loss = 'BinaryCrossentropy', optimizer= 'Adam')

In [ ]:
model.fit(X , Y, epochs = 40)

Epoch 1/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.6152
Epoch 2/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.6108
Epoch 3/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.6073
Epoch 4/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.6044
Epoch 5/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.6011
Epoch 6/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.5978
Epoch 7/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.5948
Epoch 8/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.5918
Epoch 9/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.5900
Epoch 10/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.5871
Epoch 11/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.5846
Epoch 12/40
1606/1606 [==============================] - 2s 1ms/step - loss: 0.5830
E

In [ ]:
lstm_model = tf.keras.Sequential()
lstm_model.add( tf.keras.layers.InputLayer( input_shape=X_LSTM.shape[1:] ) )
lstm_model.add(tf.keras.layers.LSTM(25, activation='relu'))
lstm_model.add(tf.keras.layers.Dense(1, activation='softmax'))

In [ ]:
lstm_model.compile( loss = 'BinaryCrossentropy', optimizer= 'Adam')

In [ ]:
lstm_model.fit(X_LSTM , Y, epochs = 4)

Epoch 1/4
 814/1606 [==============>...............] - ETA: 40s - loss: 0.6784

In [ ]:
file['X'][:1].values

array(['coronavirus origin:Monophyletic Relationship between Severe Acute Respiratory Syndrome Coronavirus and Group 2 Coronaviruses'],
      dtype=object)

In [ ]:
a = np.array([[1,2,3], [4,5,6]])
b = np.reshape(a, (2,3,1))

In [ ]:
print(a)
print(b)

[[1 2 3]
 [4 5 6]]
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]]


In [ ]:
a.shape

(2, 3)

In [ ]:
b.shape

(2, 3, 1)

In [ ]:
c= np.reshape(a, a.shape + (1,))
c.shape

(2, 3, 1)